In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam

In [3]:
# Veri setinin bulunduğu dizin
data_directory = '/content/drive/MyDrive/Colab Notebooks/Pneumonia/X-Ray'

In [4]:
# Önceden eğitilmiş modelin yüklenmesi
pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


58889256/58889256 [==============================] - 3s 0us/step


In [5]:
# Önceden eğitilmiş modelin üzerine özel katmanların eklenmesi
model = Sequential()
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [6]:
# Önceden eğitilmiş katmanların dondurulması
pretrained_model.trainable = False


In [7]:
# Modelin derlenmesi
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [8]:
# Veri artırma (data augmentation) işlemi
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


In [9]:
# Veri setinin yüklenmesi
train_generator = datagen.flow_from_directory(
        data_directory,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary',
        subset='training')

validation_generator = datagen.flow_from_directory(
        data_directory,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary',
        subset='validation')


Found 12934 images belonging to 2 classes.
Found 3233 images belonging to 2 classes.


In [10]:
# Modelin eğitimi
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size)


Epoch 1/10
404/404 [==============================] - 3857s 9s/step - loss: 0.5448 - accuracy: 0.7333 - val_loss: 0.5072 - val_accuracy: 0.7597
Epoch 2/10
404/404 [==============================] - 99s 246ms/step - loss: 0.4853 - accuracy: 0.7723 - val_loss: 0.4932 - val_accuracy: 0.7714
Epoch 3/10
404/404 [==============================] - 101s 249ms/step - loss: 0.4556 - accuracy: 0.7956 - val_loss: 0.4755 - val_accuracy: 0.7869
Epoch 4/10
404/404 [==============================] - 101s 250ms/step - loss: 0.4290 - accuracy: 0.8053 - val_loss: 0.4660 - val_accuracy: 0.7955
Epoch 5/10
404/404 [==============================] - 99s 245ms/step - loss: 0.4085 - accuracy: 0.8175 - val_loss: 0.4615 - val_accuracy: 0.7943
Epoch 6/10
404/404 [==============================] - 100s 248ms/step - loss: 0.3849 - accuracy: 0.8328 - val_loss: 0.4302 - val_accuracy: 0.8138
Epoch 7/10
404/404 [==============================] - 101s 250ms/step - loss: 0.3585 - accuracy: 0.8443 - val_loss: 0.4182 - val

In [11]:
# Test verisi için sadece rescale (ölçeklendirme) işlemi yapma
test_generator = datagen.flow_from_directory(
        data_directory,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary',
        shuffle=False)


Found 16167 images belonging to 2 classes.


In [12]:
# Test seti üzerinde modelin performansını değerlendirme
eval_result = model.evaluate(test_generator)
print("Test doğruluk oranı:", eval_result[1])  # Test doğruluk oranını yazdırma


506/506 [==============================] - 107s 212ms/step - loss: 0.2842 - accuracy: 0.8880
Test doğruluk oranı: 0.8880435228347778
